In [12]:
#LIBRERIAS 
import pandas as pd 
import numpy as np 
import plotly.express as px   #para mejorar las graficas 
import matplotlib.pyplot as plt 
import tensorflow as tf
import seaborn as sns
import plotly.graph_objects as go
import tensorflow
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split as tts
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.neighbors import KNeighborsRegressor as KNr
from sklearn.metrics import recall_score,precision_score
from sklearn.metrics import recall_score, precision_score, confusion_matrix #clasificacion binaria
from sklearn.metrics import multilabel_confusion_matrix #multivariable
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import r2_score , mean_squared_error
from pandas.core.common import random_state
from sklearn import neighbors
"""
GRUPO Nº:

  - Mondragón Payac Billy 
  - Del Carmen Rangel Renzo Daniel 
  - Panta Panta Jimy Aldair 
  - Alberca Elera Gustavo Alonso 
  - Manuel Amaya Vega
"""

df_insurance =""
#Menu Principal Inicial
def main():
    print ("\nMODELOS DE MACHIN LEARNING, PARA EL DATASET INSURANCE:")
    print ("\t0 - GRAFICAS-ANALISIS DE DATOS ")
    print ("\t1 - VISUALIZACION DEL PROCESAMIENTO REALIZADO")
    print ("\t2 - ALGORITMO KNR ")
    print ("\t3 - ALGORITMO RANDOM FOREST")
    print ("\t4 - ALGORITMO LASSO Y LINEARMODEL ")
    print ("\t5 - ALGORITMO SGD")
    print ("\t6 - PROGRAMA DE PREDICCIÓN")
    print ("\t7 - SALIR")

def ruta_dataframe():
  global df_insurance
  df_insurance = pd.read_csv("/content/insurance.csv")
  print("")
  print(10*"*","DATAFRAME INSURANCE","*"*10)
  print("")
  display(df_insurance.head())


def procesamiento_dataframe():
  global df_insurance
  #para volver los nan en 0 
  #df_insurance = df_insurance.fillna(0)
  #Usando laberEncoder para el dataset insurance
  labelencoder = LabelEncoder()
  labelencoder.fit_transform(df_insurance['sex'])
  labelencoder.fit_transform(df_insurance['smoker'])
  labelencoder.fit_transform(df_insurance['region'])

  df_insurance['sex'] = df_insurance['sex'].astype('category')
  df_insurance['smoker'] = df_insurance['smoker'].astype('category')
  df_insurance['region'] = df_insurance['region'].astype('category')

  df_insurance['sex'] = df_insurance['sex'].cat.codes
  df_insurance['smoker'] = df_insurance['smoker'].cat.codes
  df_insurance['region'] = df_insurance['region'].cat.codes
  print("")
  print(10*"*","DATAFRAME INSURANCE APLICANDO LABEL ENCONDER","*"*10)
  print("")
  display(df_insurance)

  #Se aplica Estandarizacion para scalar la data 
  
# Escalamiento
def escalamiento():
  global df_insurance
  global df_insurance_scalado
  scaler_ins = MMS()
  scaler_ins.fit(df_insurance)
  df_insurance = scaler_ins.fit_transform(df_insurance)
  #print(df_insurance)
  df_insurance_scalado= pd.DataFrame(df_insurance, columns=['age','sex','bmi','children','smoker','region','charges'])
  print("")
  print(10*"*",'DATAFRAME,POST APLICATE MINMAXESCALER',"*"*10)
  print("")
  display(df_insurance_scalado) 
  print("")
  #print("\nSe tiene la data escalada:")

def grafica_escal():
  global df_insurance_scalado
  df_insurance = pd.read_csv("/content/insurance.csv")
  ch_s = np.array(df_insurance['charges']) 
  ch_s = np.reshape(ch_s,(len(ch_s),1)) 
  #print(np.shape(ch_s)) 
  ch_s2 = np.array(df_insurance_scalado['charges']) 
  ch_s2 = np.reshape(ch_s2,(len(ch_s2),1)) 
  #print(np.shape(ch_s2))
  fig,ax = plt.subplots(1,2)
  sns.histplot(ch_s,ax=ax[0])
  ax[0].set_title("Original")
  sns.histplot(ch_s2,ax=ax[1])
  ax[1].set_title("scaled data")
  print(5*"*",'GRAFICA DEL ESCALAMIENTO A LA ETIQUETA CHARGES',"*"*5)
  print("")
  plt.show()

def graficas_analisis():
  df_insurance = pd.read_csv("/content/insurance.csv")
  #Análisis de la data
  descripcion = df_insurance.describe() #Análisis general de la data
  print(descripcion)
  print("")
  #Análisis de las gráficas
  fig1 = px.scatter(df_insurance, x = 'children', y = 'charges')
  fig1.show()

  #Conclusión: Las personas con mayor cantidad de hijos parecen tener un menor gasto en seguro de salud, sin embargo no es definitivo que
  #ambos valores estén relacionados

  fig2 = px.scatter(df_insurance, x = 'region', y = 'charges')
  fig2.show()
  #Conclusión: La región del cliente no afecta o afecta muy poco a los cargos del seguro

  fig3 = px.scatter(df_insurance, x = 'bmi', y = 'charges', color = 'smoker', size = 'children')
  fig3.show()
  #Conclusión: Los hábitos de los fumadores tienen un gran efecto en el costo de seguro,
  #Las personas con un BMI menor a 30 tienen un gasto menor a 30 k, incluso si son fumadores, sin embargo,
  # las personas con un bmi mayor a 25, y fumadores,
  # tienden a tener un gasto mucho más alto que las no fumadoras o con bmi menor a 30

  fig4 = px.scatter(df_insurance, x = 'age', y = 'charges', color = 'smoker', size = 'bmi')
  fig4.show()

  #Conclusión: La edad, los hábitos fumadores, la cantidad de hijos y el bmi son los factores con mayor influencia en el costo
  #Existen dos grupos claramente definidos, el de no fumadores con un menor costo, y de fumadores, en la parte superior
  #con un gran costo de seguro 

def algoritmo_knr():
    global df_insurance_scalado
    print(10*"*","ALGORITMO KNR PARA LA DATA INSURANCE","*"*10)
    print("")
    #definimos nuestro features y nuestro target
    X_featur2 = np.array(df_insurance_scalado.drop(columns=['charges']))
    ytarget2 = np.array(df_insurance_scalado['charges'])
    ytarget2 = np.reshape(ytarget2,(len(ytarget2),1,))

    #print(np.shape(X_featur2))
    #print(np.shape(ytarget2))

    Xtrain2,Xtest2,ytrain2,ytest2 = tts(X_featur2,ytarget2)
    r2train=[]
    r2val=[]
    for k in [1,3,5,7]:
      modeloknnr=KNr(n_neighbors=k)
      modeloknnr.fit(Xtrain2,ytrain2)
      ypredtest2 = modeloknnr.predict(Xtest2) #para el test
      ypredtrain2 = modeloknnr.predict(Xtrain2) #para entrenamiento
      r2modelknntrain=r2_score(ytrain2,ypredtrain2)
      r2train.append(round(r2modelknntrain,4))
      r2modelknntest=r2_score(ytest2,ypredtest2)
      r2val.append(round(r2modelknntest,4))
    print("Mis resultados de entrenamiento para K =[1,3,5,7]:")
    print(r2train)
    print("-"*50)
    print("Mis resultados de validacion para K =[1,3,5,7]:")
    print(r2val)
    print("\n")
  
def random_fores():
  global df_insurance_scalado
  print(10*"*","ALGORITMO RAMDON FOREST PARA LA DATA INSURANCE","*"*10)
  #definimos nuestro features y nuestro target
  X_featur1 = np.array(df_insurance_scalado.drop(columns=['charges']))
  ytarget1 = np.array(df_insurance_scalado['charges'])
  #print(np.shape(X_featur1))
  ytarget1 = np.reshape(ytarget1,(len(ytarget1),1,))
  #print(np.shape(ytarget1))
  #Se divide la data:
  Xtrain1,Xtest1,ytrain1,ytest1 = tts(X_featur1,ytarget1)
  #generamos el modelo RANDOM FOREST

  modelRF = RFR(n_estimators=100)
  modelRF.fit(Xtrain1,ytrain1.ravel())

  ypredtest1 = modelRF.predict(Xtest1) 
  ypredtrain1 = modelRF.predict(Xtrain1)
  r2RF_insurance=r2_score(ytest1,ypredtest1) 
  r2RT_insurance=r2_score(ytrain1,ypredtrain1)
  print("\nResultados de entrenamiento:")
  print(r2RT_insurance)
  print("-"*50)
  print("Resultados de Validacion:")
  print(r2RF_insurance)
  print("\n**De lo observado se dice que el modelo esta correctamente generalizado\n")


#3ER MODELO: Aplicando Modelo lineal de Regresion a nuestra data escalada_insurance, 
def ridge_linear():
  global df_insurance_scalado
  print(10*"*","ALGORITMO RIDGE Y MODEL LINEAR PARA LA DATA INSURANCE","*"*10)
  X_featur3 = np.array(df_insurance_scalado.drop(columns=['charges']))
  ytarget3 = np.array(df_insurance_scalado['charges'])
  ytarget3 = np.reshape(ytarget3,(len(ytarget3),1,))

  modelRidge = Ridge(alpha = 1)
  modelLinear = LinearRegression()
  Xtrain3,Xtest3,ytrain3,ytest3 = tts(X_featur3,ytarget3)

  r2listt = []
  r2listpr = []
  for i in [modelRidge,modelLinear]:
    i.fit(Xtrain3,ytrain3)
    ytestpred3 = i.predict(Xtest3)
    ytrainpred3 = i.predict(Xtrain3)
    r2t = r2_score(ytrain3,ytrainpred3) 
    r2listt.append(r2t)
    r2test = r2_score(ytest3,ytestpred3)
    r2listpr.append(r2test)

  print("\nResultado para entrenamiento: \n",r2listt)
  print("Resultado para validación: \n",r2listpr)

#4TO MODELO, APLICANDO EL MODELO SGD PARA DE LA DATA INSURANCE
def SGD():
  global df_insurance_scalado
  print(10*"*","ALGORITMO SDG PARA LA DATA INSURANCE","*"*10)
  #from sklearn.ensemble import GradientBoostingRegressor as SGD
  X_featur4= np.array(df_insurance_scalado.drop(columns=['charges']))
  ytarget4 = np.array(df_insurance_scalado['charges'])
  ytarget4 = np.reshape(ytarget4,(len(ytarget4),1,))
  Xtrain4,Xtest4,ytrain4,ytest4 = tts(X_featur4,ytarget4)
  modelo_sgd = GradientBoostingRegressor(learning_rate = 0.1,max_depth=3)
  modelo_sgd.fit(Xtrain4,ytrain4.ravel())
  predtrain4 = modelo_sgd.predict(Xtrain4)
  r2train4 = r2_score(ytrain4, predtrain4)
  print("\nResultado para entrenamiento: \n",r2train4)
  predval4 = modelo_sgd.predict(Xtest4)
  r2val4 = r2_score(ytest4, predval4)
  print("Resultado para validación: \n",r2val4)
  print("\n")


#Programa en terminal que predice el valor 'charges'

def Programa():
  print(10*"*","PROGRAMA DE PREDICCION","*"*10)
  print("")
  age = int(input("Ingrese su edad: "))
  sex = int(input("Ingrese si es hombre (1) o mujer (0): "))
  bmi = float(input("Ingrese su bmi: "))
  children = int(input("Ingrese el número de hijos: "))
  smoker = int(input("Ingrese si usted es fumador (1) o no fumador (0): "))
  region = int(input("Ingrese a qué región pertenece, southwest (3), southeast (2), northwest (1), northeast (0): "))
  df_insurance_new = pd.read_csv("/content/insurance.csv")

  labelencoder = LabelEncoder()
  labelencoder.fit_transform(df_insurance_new['sex'])
  labelencoder.fit_transform(df_insurance_new['smoker'])
  labelencoder.fit_transform(df_insurance_new['region'])

  df_insurance_new['sex'] = df_insurance_new['sex'].astype('category')
  df_insurance_new['smoker'] = df_insurance_new['smoker'].astype('category')
  df_insurance_new['region'] = df_insurance_new['region'].astype('category')

  df_insurance_new['sex'] = df_insurance_new['sex'].cat.codes
  df_insurance_new['smoker'] = df_insurance_new['smoker'].cat.codes
  df_insurance_new['region'] = df_insurance_new['region'].cat.codes

  X_features_programa= np.array(df_insurance_new.drop(columns=['charges']))
  ytarget_programa = np.array(df_insurance_new['charges'])
  ytarget_programa = np.reshape(ytarget_programa,(len(ytarget_programa),1,))
  Xtrain,Xtest,ytrain,ytest = tts(X_features_programa,ytarget_programa)
  modelo_sgd = GradientBoostingRegressor(learning_rate = 0.1,max_depth=3)
  modelo_sgd.fit(Xtrain,ytrain.ravel())
  charges = modelo_sgd.predict([[age, sex, bmi, children, smoker, region]])
  print("**El valor predecido para 'charges', es:",np.round(charges,3))

#BUCLE QUE MUESTRA EN PANTALLA EL MAIN() INICIAL
while True:
  main()
  eleccion=input("Inserta una Opción: ",)
  print("")
  if eleccion == "0":
        graficas_analisis()
  elif eleccion == "1":
        ruta_dataframe()
        procesamiento_dataframe()
        escalamiento()
        grafica_escal()
  elif eleccion == "2":
        algoritmo_knr()
  elif eleccion=="3":
        random_fores()
  elif eleccion=="4":
        ridge_linear()
  elif eleccion=="5":
        SGD()
  elif eleccion == "6":
        Programa()
  elif eleccion=="7":
        print("GRACIAS POR SU TIEMPO")
        break      
  else:
        print("NO INGRESO UNA OPCION CORRECTA")


MODELOS DE MACHIN LEARNING, PARA EL DATASET INSURANCE:
	0 - GRAFICAS-ANALISIS DE DATOS 
	1 - VISUALIZACION DEL PROCESAMIENTO REALIZADO
	2 - ALGORITMO KNR 
	3 - ALGORITMO RANDOM FOREST
	4 - ALGORITMO LASSO Y LINEARMODEL 
	5 - ALGORITMO SGD
	6 - PROGRAMA DE PREDICCIÓN
	7 - Salir


KeyboardInterrupt: ignored